# 8.4. Recurrent Neural Networks
In Section 8.3 we introduced  n -gram models, where the conditional probability of word  xt  at time step  t  only depends on the  n−1  previous words. If we want to incorporate the possible effect of words earlier than time step  t−(n−1)  on  xt , we need to increase  n . However, the number of model parameters would also increase exponentially with it, as we need to store  |V|n  numbers for a vocabulary set  V . Hence, rather than modeling  P(xt∣xt−1,…,xt−n+1)  it is preferable to use a latent variable model:

P(xt∣xt−1,…,x1)≈P(xt∣ht−1),
 
where  ht−1  is a hidden state (also known as a hidden variable) that stores the sequence information up to time step  t−1 . In general, the hidden state at any time step  t  could be computed based on both the current input  xt  and the previous hidden state  ht−1 :

ht=f(xt,ht−1).
 
For a sufficiently powerful function  f  in (8.4.2), the latent variable model is not an approximation. After all,  ht  may simply store all the data it has observed so far. However, it could potentially make both computation and storage expensive.

Recall that we have discussed hidden layers with hidden units in Section 4. It is noteworthy that hidden layers and hidden states refer to two very different concepts. Hidden layers are, as explained, layers that are hidden from view on the path from input to output. Hidden states are technically speaking inputs to whatever we do at a given step, and they can only be computed by looking at data at previous time steps.

Recurrent neural networks (RNNs) are neural networks with hidden states. Before introducing the RNN model, we first revisit the MLP model introduced in Section 4.1.

## 8.4.1. Neural Networks without Hidden States
Let us take a look at an MLP with a single hidden layer. Let the hidden layer’s activation function be  ϕ . Given a minibatch of examples  X∈Rn×d  with batch size  n  and  d  inputs, the hidden layer’s output  H∈Rn×h  is calculated as

H=ϕ(XWxh+bh).
 
In (8.4.3), we have the weight parameter  Wxh∈Rd×h , the bias parameter  bh∈R1×h , and the number of hidden units  h , for the hidden layer. Thus, broadcasting (see Section 2.1.3) is applied during the summation. Next, the hidden variable  H  is used as the input of the output layer. The output layer is given by

O=HWhq+bq,
 
where  O∈Rn×q  is the output variable,  Whq∈Rh×q  is the weight parameter, and  bq∈R1×q  is the bias parameter of the output layer. If it is a classification problem, we can use  softmax(O)  to compute the probability distribution of the output categories.

This is entirely analogous to the regression problem we solved previously in Section 8.1, hence we omit details. Suffice it to say that we can pick feature-label pairs at random and learn the parameters of our network via automatic differentiation and stochastic gradient descent.

## 8.4.2. Recurrent Neural Networks with Hidden States

In [1]:
import torch
from d2l import torch as d2l

X, W_xh = torch.normal(0, 1, (3, 1)), torch.normal(0, 1, (1, 4))
H, W_hh = torch.normal(0, 1, (3, 4)), torch.normal(0, 1, (4, 4))
torch.matmul(X, W_xh) + torch.matmul(H, W_hh)

tensor([[ 0.0413,  0.0132, -3.4491,  0.4877],
        [-0.8744, -0.9286,  0.5438,  0.5912],
        [ 0.1699, -0.6499, -1.3896,  0.2845]])

In [2]:
torch.matmul(torch.cat((X, H), 1), torch.cat((W_xh, W_hh), 0))

tensor([[ 0.0413,  0.0132, -3.4491,  0.4877],
        [-0.8744, -0.9286,  0.5438,  0.5912],
        [ 0.1699, -0.6499, -1.3896,  0.2845]])

## 8.4.3. RNN-based Character-Level Language Models

Recall that for language modeling in Section 8.3, we aim to predict the next token based on the current and past tokens, thus we shift the original sequence by one token as the labels. Bengio et al. first proposed to use a neural network for language modeling [Bengio et al., 2003]. In the following we illustrate how RNNs can be used to build a language model. Let the minibatch size be one, and the sequence of the text be “machine”. To simplify training in subsequent sections, we tokenize text into characters rather than words and consider a character-level language model. Fig. 8.4.2 demonstrates how to predict the next character based on the current and previous characters via an RNN for character-level language modeling.

During the training process, we run a softmax operation on the output from the output layer for each time step, and then use the cross-entropy loss to compute the error between the model output and the label. Due to the recurrent computation of the hidden state in the hidden layer, the output of time step 3 in Fig. 8.4.2,  O3 , is determined by the text sequence “m”, “a”, and “c”. Since the next character of the sequence in the training data is “h”, the loss of time step 3 will depend on the probability distribution of the next character generated based on the feature sequence “m”, “a”, “c” and the label “h” of this time step.

In practice, each token is represented by a  d -dimensional vector, and we use a batch size  n>1 . Therefore, the input  Xt  at time step  t  will be a  n×d  matrix, which is identical to what we discussed in Section 8.4.2.

## 8.4.4. Perplexity
Last, let us discuss about how to measure the language model quality, which will be used to evaluate our RNN-based models in the subsequent sections. One way is to check how surprising the text is. A good language model is able to predict with high-accuracy tokens that what we will see next. Consider the following continuations of the phrase “It is raining”, as proposed by different language models:

“It is raining outside”

“It is raining banana tree”

“It is raining piouw;kcj pwepoiut”

In terms of quality, example 1 is clearly the best. The words are sensible and logically coherent. While it might not quite accurately reflect which word follows semantically (“in San Francisco” and “in winter” would have been perfectly reasonable extensions), the model is able to capture which kind of word follows. Example 2 is considerably worse by producing a nonsensical extension. Nonetheless, at least the model has learned how to spell words and some degree of correlation between words. Last, example 3 indicates a poorly trained model that does not fit data properly.

We might measure the quality of the model by computing the likelihood of the sequence. Unfortunately this is a number that is hard to understand and difficult to compare. After all, shorter sequences are much more likely to occur than the longer ones, hence evaluating the model on Tolstoy’s magnum opus War and Peace will inevitably produce a much smaller likelihood than, say, on Saint-Exupery’s novella The Little Prince. What is missing is the equivalent of an average.

Information theory comes handy here. We have defined entropy, surprisal, and cross-entropy when we introduced the softmax regression (Section 3.4.7) and more of information theory is discussed in the online appendix on information theory. If we want to compress text, we can ask about predicting the next token given the current set of tokens. A better language model should allow us to predict the next token more accurately. Thus, it should allow us to spend fewer bits in compressing the sequence. So we can measure it by the cross-entropy loss averaged over all the  n  tokens of a sequence:

1n∑t=1n−logP(xt∣xt−1,…,x1),
 
where  P  is given by a language model and  xt  is the actual token observed at time step  t  from the sequence. This makes the performance on documents of different lengths comparable. For historical reasons, scientists in natural language processing prefer to use a quantity called perplexity. In a nutshell, it is the exponential of (8.4.7):

exp(−1n∑t=1nlogP(xt∣xt−1,…,x1)).
 
Perplexity can be best understood as the harmonic mean of the number of real choices that we have when deciding which token to pick next. Let us look at a number of cases:

In the best case scenario, the model always perfectly estimates the probability of the label token as 1. In this case the perplexity of the model is 1.

In the worst case scenario, the model always predicts the probability of the label token as 0. In this situation, the perplexity is positive infinity.

At the baseline, the model predicts a uniform distribution over all the available tokens of the vocabulary. In this case, the perplexity equals the number of unique tokens of the vocabulary. In fact, if we were to store the sequence without any compression, this would be the best we could do to encode it. Hence, this provides a nontrivial upper bound that any useful model must beat.

In the following sections, we will implement RNNs for character-level language models and use perplexity to evaluate such models.

## 8.4.5. Summary
A neural network that uses recurrent computation for hidden states is called a recurrent neural network (RNN).

The hidden state of an RNN can capture historical information of the sequence up to the current time step.

The number of RNN model parameters does not grow as the number of time steps increases.

We can create character-level language models using an RNN.

We can use perplexity to evaluate the quality of language models.